# Ch3 LLM, RAG 개인 도전과제

### Goal: LangSmith의 Prompt Library를 참고하여 prompt engineering을 수행

##### 1. 사용환경 준비

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"  # 라이브러리끼리의 충돌을 해결

##### 2. 모델 로드하기

In [2]:
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

##### 3. 문서 로드하기

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("files/인공지능산업최신동향_2024년_11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

##### 4. 문서 청크로 나누기

**RecursiveCharacterTextSplitter**
- 주어진 문자 목록의 순서대로 청크가 충분히 작아질 때까지 재귀적으로 텍스트를 분할하는 클래스
- 문자 목록을 매개변수로 받아 동작
- 기본 문자 목록: \["\n\n", "\n", " ", ""\] (default)
- 텍스트를 재귀적으로 분할하여 의미적으로 관련있는 텍스트 조각들이 같이 있도록 하는 목적으로 설계됨

---
- `chunk_size`: 각 청크의 최대 길이
- `chunk_overlap`: 인접한 청크 사이에 중복으로 포함될 문자의 수
- `length_function`: 청크의 길이를 계산하는 함수
- `is_separator_regex`: 구분자로 정규식을 사용할지 여부를 설정

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

##### 5. 벡터 임베딩 생성

In [5]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

##### 6. 벡터 스토어 생성

In [6]:
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

##### 7. FAISS를 Retriever로 변환

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

### 도전 구현 과제

##### i. LangSmith의 Prompt Library 를 참고하여 프롬프트를 3개 이상 아래와 같은 파일 구조로 저장해주세요. 
``` bash
.
├── main.jupynb
└── Prompts/
    ├── prompt1.txt
    ├── prompt2.txt
    └── prompt3.txt
```

In [37]:
# LangChain library에서 prompt, model, chain, agent 등을 불러오는 라이브러리
from langchain import hub

# Prompts 디렉토리가 없다면 Prompts 디렉토리 만들기
os.makedirs("Prompts", exist_ok=True)

# hub.pull() 함수로 LangChain library에서 prompt 가져오기
prompt1 = hub.pull("rlm/rag-prompt")
prompt2 = hub.pull("daethyra/rag-prompt")
prompt3 = hub.pull("godk/korean-rag")

# 가져온 prompt들을 prompts list에 저장
prompts = [prompt1, prompt2, prompt3]

# Prompts 디렉토리에 prompt 저장
for i in range(1, 4):
    # 파일 경로를 .\Prompts\prompt.txt 형태로 설정
    file_path = os.path.join(".\Prompts", f"prompt{i}.txt")
    
    # format()으로 ChatPromptTemplate을 str 타입으로 변환
    prompt = prompts[i-1].format(question="{question}", context="{context}")
    
    # 설정한 파일 경로로 변환한 prompt.txt 저장
    with open(file_path, 'w') as f:
        f.write(prompt)

d:\Anaconda3_envs\SpartaProjects\Personal_Project\project2\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
d:\Anaconda3_envs\SpartaProjects\Personal_Project\project2\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
d:\Anaconda3_envs\SpartaProjects\Personal_Project\project2\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


##### ii. 각 프롬프트를 외부에서 불러와서 실행할 수 있도록 코드를 고쳐주세요. 

In [38]:
prompts = {}
for i in range(1, 4):
    file_path = os.path.join(".\Prompts", f"prompt{i}.txt")
    with open(file_path, 'r') as f:
        prompts[f"prompt{i}"] = f.read()
        print(prompts[f"prompt{i}"], end="\n\n\n")

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


System: You are an expert AI on a question and answer task. 
Use the "Following Context" when answering the question. If you don't know the answer, reply to the "Following Text" in the header and answer to the best of your knowledge, or if you do know the answer, answer without the "Following Text". If a question is asked in Korean, translate it to English and always answer in Korean.
Following Text: "주

##### iii.실행 결과는 자동으로 Results 디렉토리에 저장되어야 합니다. 이때, 실험 결과 파일 이름은 실험에 쓰인 프롬프트의 이름과 timestamp을 포함해야합니다.
```bash
.
├── main.jupynb
└── Prompts/
    ├── prompt1.txt
    ├── prompt2.txt
    └── prompt3.txt
└── Results/
    ├── prompt1_result_1731314042.txt
    ├── prompt2_result_1731314050.txt
    └── prompt3_result_1731314050.txt
```

In [39]:
from langchain_core.runnables import RunnablePassthrough

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}
    
# Results 디렉토리가 없다면 Results 디렉토리 만들기
os.makedirs("Results", exist_ok=True)


In [41]:
from langchain_core.prompts import PromptTemplate
import time

for i in range(1, 4):
    prompt_template = prompts[f"prompt{i}"]
    prompt = PromptTemplate.from_template(prompt_template)
    
    # RAG 체인에서 각 단계마다 DebugPassThrough 추가
    rag_chain_debug = {
        "context": retriever,  # 컨텍스트를 가져오는 retriever
        "question": DebugPassThrough()  # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
    }  | DebugPassThrough() | ContextToText() | prompt | model
        
    print("========================")
    query = input("질문을 입력하세요: ")
    response = rag_chain_debug.invoke(query)
    print("Final Response:")
    print(response.content)
    
    # timestamp 생성
    timestamp = str(time.time()).split('.')[0]
    
    # 파일 경로를 .\Results\prompt_result_timestamp.txt 형태로 설정
    file_path = os.path.join(".\Results", f"prompt{i}_result_{timestamp}.txt")
    
    # 설정한 파일 경로로 prompt_result_timestamp.txt 저장
    with open(file_path, 'w') as f:
        f.write(response.content)

Debug Output: 일본의 AI연구소에 대해 알려줘
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 1}, page_content='AI 연구에서 국제협력을 위한 ‘글로벌 AI 연구 의제’ 발표························13   ▹ 일본 AI안전연구소, AI 안전성에 대한 평가 관점 가이드')], 'question': '일본의 AI연구소에 대해 알려줘'}
Final Response:
일본에는 AI 안전 연구소가 있으며, 이 연구소는 AI 안전성에 대한 평가 관점 가이드를 제공합니다. 또한, 일본은 AI 연구에서 국제 협력을 위한 '글로벌 AI 연구 의제'를 발표했습니다. 이를 통해 AI 기술의 안전성과 발전을 동시에 추구하고 있습니다.
Debug Output: 한국의 카카오가 선보인 최신 AI 서비스가 뭐지?
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 13}, page_content='등의 단어를 조합한 카나나는 ‘가장 나다운 AI’를 의미∙카카오는 동 브랜드를 자사가 개발하는 주요 AI 모델과 신규 서비스의 이름에 두루 사용할 계획으로, AI 메이트 서비스')], 'question': '한국의 카카오가 선보인 최신 AI 서비스가 뭐지?'}
Final Response:
한국의 카카오는 '카나나'라는 최신 AI 서비스를 선보였습니다. 이 서비스는 '가장 나다운 AI'를 의미하며, 카카오는 이를 자사가 개발하는 주요 AI 모델과 신규 서비스의 이름으로 사용할 계획입니다.
Debug Output: 2024년에 있는 주요 행사들에 대해 요약해줘
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_1